# November 2023 General Election Mayoral Analysis

Analysis of the November 2023 results for the Bellingham mayoral race in Whatcom County between Seth Fleetwood and Kim Lund

## Data Prep Instructions
This procedure is for background only.  Data files in repo have already been prepped

1. Get shape files of Precincts from Whatcom County https://documents.co.whatcom.wa.us/WebLink/Browse.aspx?id=2951672&dbid=100&repo=WC

2. Convert shape zip file to geojson using mapshaper.org

* Drag and drop shape zip onto mapshaper site
* Open Console and type mapshaper -proj wg84 to convert coordinates to lat/long
* Export to geojson

    Note this json file does not have id tag which is needed to link Precinct to external csv file. Workaroun is to use parameter 'featureidkey' to explicitly specify id

3. Download the election results from the Whatcom County.  https://results.vote.wa.gov/results/20231107/whatcom/precincts-126112.html
   
5. Import csv and json data to the repo

In [ ]:
mayor_voter_results_by_precinct = './data/general/20231107_whatcom_126112-precincts.csv'
mayor_voter_results_by_precinct_processed = './data/general/20231107_whatcom_126112-precincts-processed.csv'
geo_json_precinct_file = './data/precinct_map/April2023_Precinct_Splits_wgs84.json' # File path to the GeoJSON file

## Import Raw Election Results

In [ ]:
import pandas as pd

df_votes_by_precinct = pd.read_csv(mayor_voter_results_by_precinct, skiprows=1)

#Remove the totals in row 1
df_votes_by_precinct = df_votes_by_precinct.drop(0)

#Add precinct name to first column
df_votes_by_precinct.rename(columns={'Unnamed: 0': 'Precinct'}, inplace=True)

#Convert precinct column from object to integer
df_votes_by_precinct['Precinct'] = pd.to_numeric(df_votes_by_precinct['Precinct'], errors='coerce').fillna(0).astype(int)

#Add precinct totals column
df_votes_by_precinct['Totals By Precinct'] = df_votes_by_precinct.iloc[:, 1:].sum(axis=1)

df_votes_by_precinct.dtypes
df_votes_by_precinct

## Add Calculated Columns to Results

The Outcome column considers a difference of more than 

In [ ]:
df_votes_by_precinct['Percent Seth Votes'] = (df_votes_by_precinct['Seth Fleetwood'] / df_votes_by_precinct['Totals By Precinct']) * 100
df_votes_by_precinct['Percent Kim Votes'] = (df_votes_by_precinct['Kim Lund'] / df_votes_by_precinct['Totals By Precinct']) * 100

df_votes_by_precinct['Percent Kim-Seth'] = ((df_votes_by_precinct['Kim Lund'] - df_votes_by_precinct['Seth Fleetwood'])/ df_votes_by_precinct['Totals By Precinct']) * 100

df_votes_by_precinct['Outcome'] = pd.cut(df_votes_by_precinct['Percent Kim-Seth'], bins=[-100, -0.5, 0.5, 100], labels=['Seth', 'Neutral', 'Kim'])

df_votes_by_precinct.to_csv(mayor_voter_results_by_precinct_processed)

df_votes_by_precinct
#df_votes_by_precinct.dtypes

## Read in Geo JSON Precinct data

In [ ]:
import json

# Reading the JSON file
with open(geo_json_precinct_file, 'r') as file:
    precinct_data = json.load(file)

# Displaying the JSON data features dictionary for each precinct
#print(precinct_data)
#precinct_data["features"][0]

# Within features is a properties dictionary which contains the precinct id
#print(precinct_data["features"][0]["properties"])

## Summary Statistics

In [ ]:
# Calculating the total votes for each candidate and overall percentages
total_votes_seth = df_votes_by_precinct['Seth Fleetwood'].sum()
total_votes_kim = df_votes_by_precinct['Kim Lund'].sum()
total_votes = df_votes_by_precinct['Totals By Precinct'].sum()

percent_votes_seth = (total_votes_seth / total_votes) * 100
percent_votes_kim = (total_votes_kim / total_votes) * 100
percent_difference = percent_votes_kim - percent_votes_seth

# Determining the overall winner
winner = "Seth Fleetwood" if total_votes_seth > total_votes_kim else "Kim Lund"

# Counting the number of precincts won by each candidate
precincts_won = df_votes_by_precinct['Outcome'].value_counts()
print(precincts_won)

election_summary_stats_seth = {
    'Total Votes': total_votes_seth,
    'Percentage': "{:.2f}".format(percent_votes_seth),  # Formats the percentage to two decimal places
    'Precincts Won': precincts_won['Seth'],
    'Org Endorsements': 8 #https://sethfleetwood.com/endorsements/
}

print(election_summary_stats_seth)

election_summary_stats_kim = {
    'Total Votes': total_votes_kim,
    'Percentage': "{:.2f}".format(percent_votes_kim),  # Formats the percentage to two decimal places
    'Precincts Won': precincts_won['Kim'],
    'Org Endorsements': 19  #https://electkimlund.com/endorsements/
}

print(election_summary_stats_kim)
print(election_summary_stats_seth)

total_votes_seth, total_votes_kim, percent_votes_seth, percent_votes_kim, percent_difference, winner

In [ ]:
# Counting the number of precincts won by each candidate
precincts_won = df_votes_by_precinct['Outcome'].value_counts()
precincts_won

## Visualize integrated precinct data
Use plotly for interactive visualization.  This graph below is good for familairinzing with the precincts and generating turf maps for door-belling.  Use mouse to zoom and pan

In [ ]:
import plotly.express as px

# Create the choropleth map
fig = px.choropleth_mapbox(df_votes_by_precinct, geojson=precinct_data, color=None,
                           locations="Precinct", featureidkey="properties.Precinct",
                           labels="Precinct",
                           center={"lat": 48.7519, "lon": -122.4787},
                           mapbox_style="carto-positron", zoom=10, opacity=0.1)

# Adjust the marker properties for boundaries
for trace in fig.data:
    trace.marker.line.width = 6
    trace.marker.line.color = 'red'

# Update layout
fig.update_layout(width=1200, height=900)  # Adjust to your preference
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

### Visualize Kim Lund Primary Raw Vote Results

In [ ]:
fig = px.choropleth_mapbox(df_votes_by_precinct, geojson=precinct_data, color="Kim Lund",
                           locations="Precinct", featureidkey="properties.Precinct",
                           center={"lat": 48.7519, "lon": -122.4787},
                           mapbox_style="carto-positron", zoom=10)

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

### Visualize Seth Fleetwood Primary Raw Vote Results

In [ ]:
fig = px.choropleth_mapbox(df_votes_by_precinct, geojson=precinct_data, color="Seth Fleetwood",
                           locations="Precinct", featureidkey="properties.Precinct",
                           center={"lat": 48.7519, "lon": -122.4787},
                           mapbox_style="carto-positron", zoom=10)

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

### Plot Seth Percent Vote of Total Precinct

In [ ]:
fig = px.choropleth_mapbox(df_votes_by_precinct, geojson=precinct_data, color="Percent Seth Votes",
                           locations="Precinct", featureidkey="properties.Precinct",
                           center={"lat": 48.7519, "lon": -122.4787},
                           mapbox_style="carto-positron", zoom=10)

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

### Plot Kim Percent Vote of Total Precinct

In [ ]:
fig = px.choropleth_mapbox(df_votes_by_precinct, geojson=precinct_data, color="Percent Kim Votes",
                           locations="Precinct", featureidkey="properties.Precinct",
                           center={"lat": 48.7519, "lon": -122.4787},
                           mapbox_style="carto-positron", zoom=10)

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

## Plot the Outcome bewteen candidates as defined above

In [ ]:
fig = px.choropleth_mapbox(df_votes_by_precinct, geojson=precinct_data, color="Outcome",
                           color_discrete_map={'Kim':'blue', 'Seth':'orange', 'Neutral': 'green'},
                           locations="Precinct", featureidkey="properties.Precinct",
                           opacity=0.6,
                           center={"lat": 48.7519, "lon": -122.4787},
                           mapbox_style="carto-positron", zoom=10)

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

## Matplotlib Analysis
More customization allowed and nicer

In [ ]:
geo_json_precinct_file

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Load the GeoJSON file into a GeoDataFrame
gdf = gpd.read_file(geo_json_precinct_file)

gdf['Precinct'] = gdf['Precinct'].astype(int)

# Plot the GeoDataFrame to visualize the precinct map
fig, ax = plt.subplots(figsize=(10, 10))
gdf.boundary.plot(ax=ax)
plt.title('Precinct Map')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

In [ ]:
# Merge the GeoDataFrame with the vote data DataFrame on the "Precinct" column
merged_gdf = gdf.merge(df_votes_by_precinct, left_on='Precinct', right_on='Precinct', how='left')
#TODO: For some reason integer columns are cast to float64

In [ ]:
# Function to annotate each plot with further adjusted vertical positioning for better centering.  The function below is the simple version but
# it does not account for spherical
def annotate_precincts(ax, data, percentage_col):
    for x, y, label, pct in zip(data.geometry.centroid.x, data.geometry.centroid.y, data['Precinct'], data[percentage_col]):
        if not pd.isna(pct):  # Only annotate if the percentage is available
            ax.text(x, y - 0.003, f"{int(label)}\n{pct:.1f}%", fontsize=6, ha='center')

import geopandas as gpd
from shapely.geometry import Point

# Updated function to account for sprehical coordinates and projection to flat geometry
def annotate_precincts_new(ax, data, percentage_col):
    # Transform to a projected CRS for accurate centroid calculation
    data_projected = data.to_crs(epsg=2163)  # Replace with suitable EPSG code.   for data in the United States, a common choice is EPSG:2163 (US National Atlas Equal Area projection). You can find a suitable EPSG code from spatialreference.org or other CRS resources.

    for x, y, label, pct in zip(data_projected.geometry.centroid.x, data_projected.geometry.centroid.y, data['Precinct'], data[percentage_col]):
        if not pd.isna(pct):  # Only annotate if the percentage is available
            # Convert the centroid coordinates back to the original CRS for plotting
            point_original_crs = gpd.GeoSeries([Point(x, y)], crs=data_projected.crs).to_crs(data.crs).iloc[0]
            ax.text(point_original_crs.x, point_original_crs.y - 0.003, f"{int(label)}\n{pct:.1f}%", fontsize=6, ha='center')

In [ ]:

# Convert the statistics dictionary to a string for display
summary_text_seth = '\n'.join([f'{key}: {value}' for key, value in election_summary_stats_seth.items()])
summary_text_kim = '\n'.join([f'{key}: {value}' for key, value in election_summary_stats_kim.items()])

print(summary_text_seth)
print(summary_text_kim)

In [ ]:
import matplotlib.colors as colors
import matplotlib.colors as mcolors

# Define a custom colormap
cmap = mcolors.LinearSegmentedColormap.from_list(
    name='custom_blue_green',
    colors=['blue', 'lightblue', 'lightgreen', 'green']
)

cmap = mcolors.LinearSegmentedColormap.from_list(
    name='custom_cool_warm',
    colors=['blue', 'lightblue', 'lightgreen', 'green']
)

# Extract 10 colors from the 'coolwarm' colormap
my_coolwarm = plt.cm.get_cmap('BrBG', 10)

# Print RGB and hex values
for i in range(my_coolwarm.N):
    rgba = my_coolwarm(i)
    # Convert RGBA to RGB
    rgb = tuple(int(255 * x) for x in rgba[:3])
    # Convert RGB to hex
    hex_color = '#{:02x}{:02x}{:02x}'.format(*rgb)
    print(f"Color {i}: RGB {rgb}, Hex {hex_color}")
    
my_colors = [my_coolwarm(i) for i in range(my_coolwarm.N)]

cmap = plt.colormaps['BrBG']

# Get a light green color from the colormap
# Adjust the value to get the desired shade of green
kim_color = cmap(0.7)  
seth_color = cmap(0.3)

#print(my_colors)

# Create a custom colormap with these colors
#cmap = mcolors.LinearSegmentedColormap.from_list(
#    name='custom_cool_warm',
#    colors=colors
#)

cmap = 'BrBG'#'PuOr' #'seismic' #'BuGn' #'viridis' #'coolwarm'

In [ ]:
# Create plots for each candidate with corrected column names and updated annotations
# Further updated zoom-in parameters
lat_min = 48.68
lat_max = 48.85
lon_min = -122.55
lon_max = -122.37

#cmap = 'coolwarm'
vmin = merged_gdf[['Percent Kim Votes', 'Percent Seth Votes']].min().min()
vmax = merged_gdf[['Percent Kim Votes', 'Percent Seth Votes']].max().max()

# Columns to plot
corrected_candidates = ['Percent Kim Votes', 'Percent Seth Votes']

#fig, axs = plt.subplots(2, 2, figsize=(18, 18))
fig, axs = plt.subplots(1, 2, figsize=(18, 8))

# Add a subtitle to the overall figure
fig.suptitle("2023 Bellingham Mayoral Race - 11/12/2023", fontsize=16)
fig.text(0.8, 0.01, 'Analysis By Gil Lund v.20231112-1', ha='center', va='bottom', fontsize=10)

# ... [your existing code for setting up the plots] ...

# Add the summary text box
# You may need to adjust the x and y coordinates and the fontsize as needed


# Flatten the array of axes for easy iteration
axes = axs.flatten()

data_precinct_border_color = '#A9A9A9'
no_data_precinct_color = '#F5F5F5' #white smoke

#Calibrate the color scale of the heatmap
#norm = colors.Normalize(vmin=-35, vmax=35)

# Create plots for each candidate
for i, candidate in enumerate(corrected_candidates):
    ax = axes[i]
    merged_gdf.boundary.plot(ax=ax, linewidth=1, color=data_precinct_border_color)
    merged_gdf.plot(column=candidate, ax=ax, legend=True,
                    legend_kwds={'label': f"{candidate.replace('Percent ', '')} by Precinct"},
                    cmap=cmap, vmin=vmin, vmax=vmax, missing_kwds={'color': no_data_precinct_color})
    ax.set_xlim([lon_min, lon_max])
    ax.set_ylim([lat_min, lat_max])
    ax.tick_params(axis='both', which='major', labelsize=8)
    ax.set_title(f"{candidate.replace('Percent ', '% ')} by Precinct")
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    print(candidate)
    if candidate == 'Percent Kim Votes':
        summary_text = summary_text_kim
    if candidate == 'Percent Seth Votes':
        summary_text = summary_text_seth
    print(summary_text)
    ax.text(0.65, 0.95, summary_text, transform=ax.transAxes, fontsize=12,
        verticalalignment='top', bbox=dict(boxstyle="round,pad=0.5", facecolor='white', edgecolor='black', alpha=0.5))
    
    # Annotate the plot with further adjusted vertical positioning
    annotate_precincts_new(ax, merged_gdf, candidate)

plt.tight_layout()

plt.savefig("2023-mayoral-general-election.png", dpi=600)
plt.show()


In [ ]:
# Create plots for each candidate with corrected column names and updated annotations
import seaborn as sns

# Columns to plot
corrected_candidates = ['Percent Kim-Seth']

vmin = merged_gdf['Percent Kim-Seth'].min()
vmax = merged_gdf['Percent Kim-Seth'].max()

# Set the overall figure size
fig = plt.figure(figsize=(20, 10))
fig.suptitle("2023 Bellingham Mayoral Race - 11/12/2023", fontsize=16)
fig.text(0.65, 0.12, 'Analysis By Gil Lund v.20231112-1', ha='center', va='bottom', fontsize=10)

# Add first subplot for ax1 - takes full height
ax1 = fig.add_subplot(121)

data_precinct_border_color = '#A9A9A9'
no_data_precinct_color = '#F5F5F5' #white smoke

data_column = 'Percent Kim-Seth'

#Calibrate the color scale of the heatmap
norm = colors.Normalize(vmin=-35, vmax=35)

merged_gdf.boundary.plot(ax=ax1, linewidth=1, color=data_precinct_border_color)
merged_gdf.plot(column=data_column, ax=ax1, legend=True, norm=norm,
                legend_kwds={'label': f"{data_column.replace('Percent ', '')} by Precinct", 'shrink': 0.7},
                cmap=cmap, vmin=vmin, vmax=vmax, missing_kwds={'color': no_data_precinct_color})

ax1.set_xlim([lon_min, lon_max])
ax1.set_ylim([lat_min, lat_max])
ax1.tick_params(axis='both', which='major', labelsize=8)
ax1.set_title('Percent Difference (Kim-Seth) by Precinct')
ax1.set_xlabel('Longitude')
ax1.set_ylabel('Latitude')
#ax1.legend(fontsize='medium')  # Adjust 'small' to 'x-small', 'medium', etc., as needed

# Continue with your plotting code for ax1

# Annotate the plot with further adjusted vertical positioning
annotate_precincts_new(ax1, merged_gdf, data_column)

# Box and whisker plot for 'Percent Kim-Seth' on the right side
#sns.boxplot(y=merged_gdf[data_column], ax=ax2, color='lightblue')
#ax2.set_title('Boxplot of Percent Kim-Seth')
#ax2.set_ylabel('Percent Difference')

ax2 = fig.add_axes([0.45, 0.2, 0.4, 0.55])  # Adjust these values as needed

#sns.violinplot(y=merged_gdf[data_column], inner='point', color='lightblue')
#sns.stripplot(y=merged_gdf[data_column], color='blue', jitter=True)
sns.swarmplot(y=merged_gdf[data_column], color='blue', size=5)

ax2.set_yticks(range(-35, 40, 5))  # This will create ticks at -35, -30, ..., 30, 35
ax2.tick_params(axis='y', labelsize=12)  # Adjust the font size as needed
ax2.grid(True, axis='y')

#print(ax2.get_xlim()[0])
#print(ax2.get_xlim()[1])
ax2.set_xlim([-1, 1])

ax2.fill_betweenx(y=[0, 35], x1=ax2.get_xlim()[0], x2=ax2.get_xlim()[1], color=kim_color, alpha=0.3)
ax2.text(ax2.get_xlim()[1], 17.5, 'Kim', color='black', verticalalignment='center', horizontalalignment='right', size=10)

ax2.fill_betweenx(y=[-35, 0], x1=ax2.get_xlim()[0], x2=ax2.get_xlim()[1], color=seth_color, alpha=0.3)
ax2.text(ax2.get_xlim()[1], -17.5, 'Seth', color='black', verticalalignment='center', horizontalalignment='right', size=10)

ax2.set_aspect(.1, adjustable='box')  # Adjust the aspect ratio and mode as needed

plt.title('Swarm Plot of Percent Difference (Kim-Seth)')
plt.ylabel('Percent Difference (Kim-Seth)')

plt.tight_layout()

plt.savefig("2023-mayoral-general-election-diff.png", dpi=600)
plt.show()